In [2]:
from split_data import split_seq_data
from load_seq_data import load_train_data
from sasrec import  SASRec

In [3]:
file_path = 'D:/PyCharm/algorithm/xunlianxingmu/ml-1m/ratings.dat'

In [4]:
train_path, val_path, test_path, meta_path = split_seq_data(file_path=file_path)

Processing lines: 100%|██████████| 1000209/1000209 [00:01<00:00, 723697.17it/s]


In [5]:
print(train_path)

D:/PyCharm/algorithm/xunlianxingmu/ml-1m\games_seq_train.txt


In [6]:
# 假设你已经有了 meta_path 的路径
with open(meta_path, 'r', encoding='utf-8') as f:
    content = f.read()
    print("Meta file content:")
    print(content)

Meta file content:
6040	3706


In [7]:
# # 调用 load_data 函数并处理返回的数据
# file_path = file_path  # 替换为实际文件路径
# neg_num = 4  # 每个正样本生成4个负样本
# max_item_num = 3706  # 假设最大物品ID为3706（根据实际情况调整）
# 
# loaded_data = load_data(file_path, neg_num, max_item_num)
# 
# # 打印部分数据以验证
# print("Sample data:")
# for i in range(min(5, len(loaded_data['user']))):  # 打印前五个样本或所有样本（如果少于5个）
#     print(f"User: {loaded_data['user'][i]}, Pos Item: {loaded_data['pos_item'][i]}, Neg Items: {loaded_data['neg_item'][i]}")

In [8]:
# 使用函数加载训练集数据
train_path = train_path # 训练集文件路径
max_item_num = 3706  # 假设最大物品ID为3706（根据实际情况调整）
seq_len = 50  # 序列长度
neg_num = 4   # 每个正样本生成4个负样本

# 加载训练数据
train_data = load_train_data(train_path, seq_len=seq_len, neg_num=neg_num, max_item_num=max_item_num, contain_user=True)

# 打印部分数据以检查是否正确加载
if train_data:
    print("Train data example:", train_data[0])
else:
    print("No train data loaded.")

Train data example: {'click_seq': [135, 130, 21, 172, 76, 117, 139, 98, 49, 133, 152, 63, 79, 114, 145, 149, 157, 91, 111, 146, 151, 156, 121, 167, 94, 118, 59, 123, 66, 120, 126, 95, 158, 161, 70, 75, 84, 119, 147, 155, 108, 138, 61, 99, 92, 127, 164, 74, 80, 131], 'pos_item': 67, 'neg_items': [2576, 209, 2750, 3182], 'user_id': 2}


In [9]:
val_path = val_path
test_path = test_path
val_data = load_train_data(val_path, seq_len=seq_len, neg_num=neg_num, max_item_num=max_item_num, contain_user=True)
test_data = load_train_data(test_path, seq_len=seq_len, neg_num=neg_num, max_item_num=max_item_num, contain_user=True)

In [10]:
import pandas as pd

# 假设 test_data 是一个包含字典的列表
df_test = pd.DataFrame(test_data)

# 查看前几行
print(df_test.head())

                                           click_seq  pos_item  \
0  [32, 23, 28, 38, 25, 37, 4, 8, 48, 1, 22, 45, ...        33   
1  [130, 21, 172, 76, 117, 139, 98, 49, 133, 152,...        73   
2  [344, 40, 196, 28, 325, 121, 314, 239, 253, 30...        19   
3  [387, 392, 420, 424, 400, 89, 105, 135, 423, 4...       154   
4  [491, 54, 28, 60, 439, 32, 455, 323, 443, 446,...        41   

                  neg_items  user_id  
0  [2440, 1080, 1732, 1805]        1  
1    [250, 3477, 606, 1046]        2  
2   [1323, 699, 2793, 1182]        5  
3  [3576, 1888, 2693, 1941]        6  
4   [3536, 2842, 2211, 756]        8  


In [11]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# 假设 max_user_num 和 max_item_num 已经通过某些方式计算得到
max_user_num = 10000  # 示例最大用户ID
max_item_num = 3706   # 示例最大物品ID

# 直接定义模型参数
model_params = {
    'item_num': 3706 + 1,          # 物品数量加1，以适应从1开始的索引
    'embed_dim': 64,               # 嵌入维度
    'seq_len': 50,                 # 序列长度，你可以根据数据集调整这个值
    'blocks': 2,                   # Transformer blocks的数量
    'num_heads': 1,                # 注意力头的数量
    'ffn_hidden_unit': 128,        # FFN隐藏单元的数量
    'dnn_dropout': 0.2,            # Dropout率
    'layer_norm_eps': 1e-6,        # Layer normalization 的小浮点数，避免除零
    'use_l2norm': False,           # 是否对用户和物品嵌入进行L2归一化
    'loss_name': "binary_cross_entropy_loss",  # 默认损失函数名
    'gamma': 0.5,                  # 如果选择 hinge_loss 作为损失函数，则可以指定 margin
    'embed_reg': 0.,               # Embedding 正则化系数
    'seed': 42                     # 随机种子
}

# 打印配置
print("Model parameters:", model_params)



Model parameters: {'item_num': 3707, 'embed_dim': 64, 'seq_len': 50, 'blocks': 2, 'num_heads': 1, 'ffn_hidden_unit': 128, 'dnn_dropout': 0.2, 'layer_norm_eps': 1e-06, 'use_l2norm': False, 'loss_name': 'binary_cross_entropy_loss', 'gamma': 0.5, 'embed_reg': 0.0, 'seed': 42}


### 创建模型实例

In [12]:
# 创建 SASRec 模型实例
sasrec_model = SASRec(**model_params)
# 设置优化器的学习率
learning_rate = 0.001  # 默认学习率
# 编译模型
sasrec_model.compile(optimizer=Adam(learning_rate=learning_rate))


In [13]:
# 打印模型结构
sasrec_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             237248      input_1[0][0]                    
                                                                 tf.reshape[0][0]                 
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
tf.math.not_equal (TFOpLambda)  (None, 50)           0           input_1[0][0]                    
______________________________________________________________________________________________

In [14]:
import time
from tensorflow.keras.optimizers import Adam  # 导入Adam优化器

# 假设 eval_pos_neg 和 SASRec 模型定义在其他文件中
from evaluate_model import  Evaluate_model # 自定义评估函数
from sasrec import SASRec  # 推荐系统模型


### 使用自定义评估函数进行训练和评估

In [15]:
#转换数据格式
from data_change import prepare_for_keras
# 定义序列长度和负样本数量
seq_len = 50  # 根据实际情况调整
neg_num = 4  # 每个正样本生成4个负样本

# 准备训练、验证和测试数据
train_data_prepared = prepare_for_keras(train_data, seq_len, neg_num)
val_data_prepared = prepare_for_keras(val_data, seq_len, neg_num)
test_data_prepared = prepare_for_keras(test_data, seq_len, neg_num)

# 现在你可以使用准备好的数据进行训练和评估

In [17]:
# 检查数据结构
print("Train data example:", train_data[0] if train_data else "No data")
print("Validation data example:", val_data[0] if val_data else "No data")
print("Test data example:", test_data[0] if test_data else "No data")

Train data example: {'click_seq': [135, 130, 21, 172, 76, 117, 139, 98, 49, 133, 152, 63, 79, 114, 145, 149, 157, 91, 111, 146, 151, 156, 121, 167, 94, 118, 59, 123, 66, 120, 126, 95, 158, 161, 70, 75, 84, 119, 147, 155, 108, 138, 61, 99, 92, 127, 164, 74, 80, 131], 'pos_item': 67, 'neg_items': [2576, 209, 2750, 3182], 'user_id': 2}
Validation data example: {'click_seq': [135, 130, 21, 172, 76, 117, 139, 98, 49, 133, 152, 63, 79, 114, 145, 149, 157, 91, 111, 146, 151, 156, 121, 167, 94, 118, 59, 123, 66, 120, 126, 95, 158, 161, 70, 75, 84, 119, 147, 155, 108, 138, 61, 99, 92, 127, 164, 74, 80, 131], 'pos_item': 67, 'neg_items': [2816, 736, 1587, 3038], 'user_id': 2}
Test data example: {'click_seq': [32, 23, 28, 38, 25, 37, 4, 8, 48, 1, 22, 45, 10, 52, 44, 42, 49, 19, 12, 15, 43, 18, 40, 46, 27, 3, 7, 20, 39, 53, 2, 14, 50, 51, 16, 21, 47, 6, 9, 13, 29, 24, 11, 17, 30, 34, 41, 5, 31, 36], 'pos_item': 33, 'neg_items': [2440, 1080, 1732, 1805], 'user_id': 1}


In [ ]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 假设 SASRec 类和其他依赖项已经定义好了，并且你已经有了 train_data, val_data, test_data

# 定义训练和评估的超参数
epochs = 5  # 训练轮数
batch_size = 128  # 批量大小
k = 10  # 评估时考虑的前K个推荐项目
seq_len = 50  # 序列长度（根据实际情况调整）
neg_num = 4  # 每个正样本生成4个负样本

# 初始化模型并编译
item_num = 3706  # 根据实际情况调整最大物品ID
embed_dim = 50  # 嵌入维度
model_params = {
    'item_num': item_num,
    'embed_dim': embed_dim,
    'seq_len': seq_len,
    'blocks': 2,
    'num_heads': 1,
    'ffn_hidden_unit': 64,
    'dnn_dropout': 0.2,
    'layer_norm_eps': 1e-6,
    'use_l2norm': True,
    'loss_name': "binary_cross_entropy_loss",
    'gamma': 0.5,
    'embed_reg': 0.,
    'seed': 42
}

model = SASRec(**model_params)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')

# 创建 TensorFlow 数据集以提高效率
def create_tf_dataset(data, batch_size):
    click_seqs = np.array([entry['click_seq'][-seq_len:] for entry in data])
    pos_items = np.array([entry['pos_item'] for entry in data])
    neg_items = np.array([entry['neg_items'][:neg_num] for entry in data])

    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'click_seq': click_seqs,
            'pos_item': pos_items,
            'neg_item': neg_items
        },
        tf.ones((len(click_seqs),))  # 假设正样本标签为1
    ))
    return dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_dataset = create_tf_dataset(train_data, batch_size)
val_dataset = create_tf_dataset(val_data, batch_size)

# 开始训练和评估
for epoch in range(1, epochs + 1):
    t1 = time.time()
    history = model.fit(
        train_dataset,
        epochs=1,
        validation_data=val_dataset,
        verbose=1  # 设置为1可以在每个epoch后输出进度条
    )
    t2 = time.time()

    # 假设 Evaluate_model 函数已经定义好，并能返回评估指标字典
    eval_dict = Evaluate_model(model, test_data, k=k)
    print('Iteration %d Fit [%.1f s], Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f' %
          (epoch, t2 - t1, time.time() - t2, eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))

# 如果需要保存模型
# model.save('sasrec_model.h5')

33/33 [==============================] - 4s 63ms/step - loss: 9.2387 - val_loss: 8.1855
Iteration 1 Fit [3.6 s], Evaluate [89.2 s]: HR = 1.0000, MRR = 0.4479, NDCG = 1.0000
33/33 [==============================] - 2s 56ms/step - loss: 7.4575 - val_loss: 6.5565
Iteration 2 Fit [1.8 s], Evaluate [88.1 s]: HR = 1.0000, MRR = 0.4291, NDCG = 1.0000
33/33 [==============================] - 2s 55ms/step - loss: 5.9661 - val_loss: 5.5074
Iteration 3 Fit [1.8 s], Evaluate [88.0 s]: HR = 1.0000, MRR = 0.4150, NDCG = 1.0000
33/33 [==============================] - 2s 58ms/step - loss: 5.0978 - val_loss: 4.8941


Columns in the dataset: ['1\t32 23 28 38 25 37 4 8 48 1 22 45 10 52 44 42 49 19 12 15 43 18 40 46 27 3 7 20 39 53 2 14 50 51 16 21 47 6 9 13 29 24 11 17 30 34 41 5 31 36 33\t978300019 978300055 978300055 978300055 978300103 978300172 978300275 978300719 978300719 978300760 978300760 978300760 978301368 978301398 978301570 978301590 978301619 978301713 978301752 978301753 978301753 978301777 978301777 978301777 978301953 978301968 978302039 978302039 978302091 978302091 978302109 978302124 978302149 978302174 978302188 978302205 978302205 978302268 978302268 978302281 978824139 978824195 978824268 978824268 978824268 978824268 978824268 978824291 978824291 978824291 978824330']
  1\t32 23 28 38 25 37 4 8 48 1 22 45 10 52 44 42 49 19 12 15 43 18 40 46 27 3 7 20 39 53 2 14 50 51 16 21 47 6 9 13 29 24 11 17 30 34 41 5 31 36 33\t978300019 978300055 978300055 978300055 978300103 978300172 978300275 978300719 978300719 978300760 978300760 978300760 978301368 978301398 978301570 978301590 9783